In [1]:
import pandas as pd 
from datasets import load_dataset, load_from_disk
import pyarrow.parquet as pq  
import time
import numpy as np

In [ ]:
parquet_path = "/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet"
table = pq.read_table(parquet_path)
clusters = table.to_pylist()
print(f"Loaded {len(clusters)} clusters into memory.")

In [ ]:
import pyarrow.parquet as pq
import random

class CachedRowGroupClusterSampler:
    def __init__(self, parquet_path):
        self.pf = pq.ParquetFile(parquet_path)
        self.num_row_groups = self.pf.num_row_groups
        self.row_group_cache = []
        self.current_rg_index = None

    def sample_cluster(self):
        # If cache is empty, load a new row group
        if not self.row_group_cache:
            # Choose a random row group
            rg_idx = random.randint(0, self.num_row_groups - 1)
            table = self.pf.read_row_group(rg_idx)
            rows = table.to_pylist()  # 1840 rows typically
            random.shuffle(rows)  # Shuffle to maintain randomness
            self.row_group_cache.extend(rows)
            self.current_rg_index = rg_idx

        # Pop one sample from cache
        row = self.row_group_cache.pop()
        return {
            "cluster_id": row["cluster_id"],
            "representative_id": row["representative_id"],
            "members": row["members"],
        }


In [ ]:
class RandomClusterSampler:
    def __init__(self, parquet_path):
        self.path = parquet_path
        
        # Read metadata once
        self.meta = pq.read_metadata(parquet_path)
        self.num_row_groups = self.meta.num_row_groups 
        
        # Open ParquetFile once 
        self.pf = pq.ParquetFile(parquet_path)

    def sample_clusters(self):
        # randomly select a row group (multiple clusters per row group)
        rg = np.random.randint(0, self.num_row_groups)

        # read only that row group 
        table = self.pf.read_row_group(rg)
        n = table.num_rows

        # Pick random row (cluster) from the row group
        cluster_idx = np.random.randint(0, n)
        row = table.slice(cluster_idx, 1)

        cluster_id = row["cluster_id"][0].as_py()
        rep_id      = row["representative_id"][0].as_py()
        members     = row["members"][0].as_py()

        return {
            "cluster_id": cluster_id,
            "representative_id": rep_id,
            "members": members
        }

In [ ]:
class InMemoryClusterSampler:
    def __init__(self, parquet_path):
        print(f"Loading Parquet file into memory (Arrow table): {parquet_path}...")
        self.table = pq.read_table(parquet_path)  
        self.num_rows = self.table.num_rows
        print(f"Loaded Arrow table with {self.num_rows} clusters.")

    def sample_cluster(self):
        idx = random.randint(0, self.num_rows - 1)
        row = {k: self.table[k][idx].as_py() for k in self.table.column_names}
        return {
            "cluster_id": row["cluster_id"],
            "representative_id": row["representative_id"],
            "members": row["members"],
        }

In [ ]:
parquet_path = "/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet"
sampeler1_start_time = time.time()
sampler1 = CachedRowGroupClusterSampler(parquet_path)
# Example usage
sampler1_samples = []
for i in range(129):
    sample = sampler1.sample_cluster()
    sampler1_samples.append(sample)
print(len(sampler1_samples))  # Print a sampled cluster
sampeler1_end_time = time.time()
print(f"Sampler1 Time taken: {sampeler1_end_time - sampeler1_start_time} seconds")

129
Sampler1 Time taken: 0.8135685920715332 seconds


In [ ]:
parquet_path = "/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet"
sampeler2_start_time = time.time()
sampler2 = RandomClusterSampler(parquet_path)
# Example usage
sampler2_samples = []
for i in range(129):
    sample = sampler2.sample_clusters()
    sampler2_samples.append(sample)
print(len(sampler2_samples))  # Print a sampled cluster
sampeler2_end_time = time.time()
print(f"Sampler1 Time taken: {sampeler2_end_time - sampeler2_start_time} seconds")

129
Sampler1 Time taken: 5.8889000415802 seconds


In [ ]:
meta = load_dataset("parquet", 
                    data_files="/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet")["train"]

Generating train split: 184146434 examples [00:20, 8911737.66 examples/s] 


In [ ]:
ds = load_dataset(
    "parquet",
    data_files="/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet",
    split="train",
    streaming=True
)


In [ ]:
stream = load_dataset(
    "parquet",
    data_files="/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet",
    streaming=True
)

In [ ]:
from datasets import Dataset

ds = Dataset.from_parquet("/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet")

Generating train split: 184146434 examples [00:18, 9966633.86 examples/s] 


In [ ]:
from datasets import load_dataset

ds = load_dataset(
    "parquet",
    data_files="/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet",
    split="train",      # required if not streaming
    streaming=False     # or just omit this arg
)

print(ds)
print(ds.features)


Dataset({
    features: ['cluster_id', 'representative_id', 'member_count_xml_file', 'member_count', 'members'],
    num_rows: 184146434
})
{'cluster_id': Value(dtype='string', id=None), 'representative_id': Value(dtype='string', id=None), 'member_count_xml_file': Value(dtype='int32', id=None), 'member_count': Value(dtype='int32', id=None), 'members': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}


In [ ]:
import pyarrow.parquet as pq
from datasets import Dataset

table = pq.read_table("/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet")
ds = Dataset(table)
print(ds)

Dataset({
    features: ['cluster_id', 'representative_id', 'member_count_xml_file', 'member_count', 'members'],
    num_rows: 184146434
})


In [ ]:
import pyarrow.parquet as pq

table = pq.read_table("/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet")
table

pyarrow.Table
cluster_id: string
representative_id: string
member_count_xml_file: int32
member_count: int32
members: list<element: string>
  child 0, element: string
----
cluster_id: [["UniRef90_UPI002E2621C6","UniRef90_UPI00358F51CD","UniRef90_UPI00398E31D8","UniRef90_A0A5A9P0L4","UniRef90_A0ABD1JBH0",...,"UniRef90_UPI00225B580A","UniRef90_UPI00254460F6","UniRef90_UPI002796F535","UniRef90_UPI002896634E","UniRef90_UPI003D7BF59D"],["UniRef90_UPI00222E980F","UniRef90_UPI0024785306","UniRef90_UPI0035A280ED","UniRef90_UPI001B351495","UniRef90_UPI00222EC2CE",...,"UniRef90_UPI00248289A7","UniRef90_UPI0012FEA33D","UniRef90_A0A8C5NYK4","UniRef90_UPI0025ADAEBD","UniRef90_UPI003BF9D085"],...,["UniRef90_A0A3M7PZG0","UniRef90_Q9BM67","UniRef90_Q9BM66","UniRef90_E9BBM2","UniRef90_Q9BLZ4",...,"UniRef90_A0A0T5Z569","UniRef90_D4J9Q7","UniRef90_A0A6A7YBE0","UniRef90_A0A0T5Z260","UniRef90_A0A5C6IVY6"],["UniRef90_Q725V0","UniRef90_A0A4R4SIP9","UniRef90_A0ABW1B1Z7","UniRef90_A0A943LZD8","UniRef90_A0ABV6HH

In [ ]:
pq.read_metadata("/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet").num_rows

184146434

In [ ]:
table.schema

cluster_id: string
representative_id: string
member_count_xml_file: int32
member_count: int32
members: list<element: string>
  child 0, element: string

In [ ]:
import pandas as pd
df = pd.read_csv("/gpfs/data/brandeslab/Data/uniref/uniref90_cluster_members.tsv", sep="\t", nrows=100)
df

,ClusterID,RepresentativeID,member_count_xml_file,member_count,MemberIDs
0,UniRef90_UPI002E2621C6,UPI002E2621C6,1,1,NaN
1,UniRef90_UPI00358F51CD,UPI00358F51CD,1,1,NaN
2,UniRef90_UPI00398E31D8,UPI00398E31D8,1,1,NaN
3,UniRef90_A0A5A9P0L4,A0A5A9P0L4_9TELE,1,1,NaN
4,UniRef90_A0ABD1JBH0,A0ABD1JBH0_9TELE,1,1,NaN
...,...,...,...,...,...
95,UniRef90_A0A6P8RG45,A0A6P8RG45_GEOSA,1,1,NaN
96,UniRef90_UPI003D69693D,UPI003D69693D,1,1,NaN
97,UniRef90_UPI003EBDD6D2,UPI003EBDD6D2,3,3,"UPI003EBC14C6,UPI003EB9A168"
98,UniRef90_A0A1V4K6M4,A0A1V4K6M4_PATFA,562,562,"A0A7K4RU14_COLPI,A0A7L4G2H3_9COLU,A0A094KAD8_A..."


In [ ]:
df.iloc[97]

ClusterID                     UniRef90_UPI003EBDD6D2
RepresentativeID                       UPI003EBDD6D2
member_count_xml_file                              3
member_count                                       3
MemberIDs                UPI003EBC14C6,UPI003EB9A168
Name: 97, dtype: object

In [ ]:
!pip install parasail

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
import parasail

In [ ]:
import sys
!{sys.executable} -m pip install parasail

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Split
from transformers import PreTrainedTokenizerFast

# Define vocabulary: ONLY standard 20 amino acids
amino_acids = list("ACDEFGHIKLMNPQRSTVWY")  # Standard 20 only
special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "-"]

# Build vocabulary
vocab = {token: i for i, token in enumerate(special_tokens + amino_acids)}

print(f"Vocab: {vocab}")
print(f"'-' ID: {vocab['-']}")

# Create tokenizer
tokenizer = Tokenizer(WordLevel(vocab=vocab, unk_token="[UNK]"))
tokenizer.pre_tokenizer = Split(pattern="", behavior="isolated")

# Wrap it
hf_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]"
)

# Test
print("\nTests:")
print("'-' converts to:", hf_tokenizer.convert_tokens_to_ids("-"))
print("Tokenization of 'ACD-GH':", hf_tokenizer("ACD-GH"))
print("Decoding:", hf_tokenizer.decode([6, 7, 8, 5, 11, 12]))
print("Vocab size:", len(hf_tokenizer))

# hf_tokenizer.save_pretrained("phylo_char_tokenizer")

Vocab: {'[PAD]': 0, '[UNK]': 1, '[CLS]': 2, '[SEP]': 3, '[MASK]': 4, '-': 5, 'A': 6, 'C': 7, 'D': 8, 'E': 9, 'F': 10, 'G': 11, 'H': 12, 'I': 13, 'K': 14, 'L': 15, 'M': 16, 'N': 17, 'P': 18, 'Q': 19, 'R': 20, 'S': 21, 'T': 22, 'V': 23, 'W': 24, 'Y': 25}
'-' ID: 5

Tests:
'-' converts to: 5
Tokenization of 'ACD-GH': {'input_ids': [6, 7, 8, 5, 11, 12], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}
Decoding: A C D - G H
Vocab size: 26


In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Split
from transformers import PreTrainedTokenizerFast

# Define vocabulary: ONLY standard 20 amino acids
amino_acids = list("ACDEFGHIKLMNPQRSTVWY")  # Standard 20 only
special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "[GAP]", "-"]


# Build vocabulary
vocab = {token: i for i, token in enumerate(special_tokens + amino_acids)}

print(f"Vocab: {vocab}")
print(f"'-' ID: {vocab['-']}")
print(f"[GAP] ID: {vocab['[GAP]']}")
print(f"[MASK] ID: {vocab['[MASK]']}")

# Create tokenizer
tokenizer = Tokenizer(WordLevel(vocab=vocab, unk_token="[UNK]"))
tokenizer.pre_tokenizer = Split(pattern="", behavior="isolated")

# Wrap it
hf_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
    additional_special_tokens=["[GAP]"]  # 👈 Register [GAP] as special
)

# Test
print("\nTests:")
print("'-' converts to:", hf_tokenizer.convert_tokens_to_ids("-"))
print("[GAP] converts to:", hf_tokenizer.convert_tokens_to_ids("[GAP]"))
print("[MASK] converts to:", hf_tokenizer.convert_tokens_to_ids("[MASK]"))
print("\n")
print("Tokenization of 'ACD-GH':", hf_tokenizer("ACD-GH", add_special_tokens=False))
print("Tokenization of  'ACD[GAP]GH':", hf_tokenizer("ACD[GAP]GH", add_special_tokens=False))
# print("Decoding:", hf_tokenizer.decode([6, 7, 8, 5, 11, 12]))
print("Vocab size:", len(hf_tokenizer))

hf_tokenizer.save_pretrained("phylo_char_tokenizer_updated")

Vocab: {'[PAD]': 0, '[UNK]': 1, '[CLS]': 2, '[SEP]': 3, '[MASK]': 4, '[GAP]': 5, '-': 6, 'A': 7, 'C': 8, 'D': 9, 'E': 10, 'F': 11, 'G': 12, 'H': 13, 'I': 14, 'K': 15, 'L': 16, 'M': 17, 'N': 18, 'P': 19, 'Q': 20, 'R': 21, 'S': 22, 'T': 23, 'V': 24, 'W': 25, 'Y': 26}
'-' ID: 6
[GAP] ID: 5
[MASK] ID: 4

Tests:
'-' converts to: 6
[GAP] converts to: 5
[MASK] converts to: 4


Tokenization of 'ACD-GH': {'input_ids': [7, 8, 9, 6, 12, 13], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}
Tokenization of  'ACD[GAP]GH': {'input_ids': [7, 8, 9, 5, 12, 13], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}
Vocab size: 27


('phylo_char_tokenizer_updated/tokenizer_config.json',
 'phylo_char_tokenizer_updated/special_tokens_map.json',
 'phylo_char_tokenizer_updated/tokenizer.json')

In [ ]:
import parasail 

In [ ]:
import sys
print(sys.executable)

/gpfs/home/as12267/.conda/envs/huggingface_bert/bin/python


In [ ]:
import sys
[p for p in sys.path if "conda" in p]

['/gpfs/home/as12267/.conda/envs/huggingface_bert/lib/python310.zip',
 '/gpfs/home/as12267/.conda/envs/huggingface_bert/lib/python3.10',
 '/gpfs/home/as12267/.conda/envs/huggingface_bert/lib/python3.10/lib-dynload',
 '/gpfs/home/as12267/.conda/envs/huggingface_bert/lib/python3.10/site-packages']

In [ ]:
import parasail
result = parasail.nw_trace_scan_16("AAAA", "AAAA", 5, 1, parasail.blosum62)

print(dir(result))

['__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_as_parameter_', '_cigar', '_traceback', '_traceback_args', 'cigar', 'end_query', 'end_ref', 'get_cigar', 'get_traceback', 'len_query', 'len_ref', 'length', 'length_col', 'length_row', 'length_table', 'matches', 'matches_col', 'matches_row', 'matches_table', 'matrix', 'pointer', 'query', 'ref', 'saturated', 'score', 'score_col', 'score_row', 'score_table', 'similar', 'similar_col', 'similar_row', 'similar_table', 'traceback']


In [ ]:
import parasail

matrix = parasail.blosum62

def align_pair(seq1, seq2):
    # global Needleman–Wunsch alignment with traceback
    result = parasail.nw_trace_scan_16(seq1, seq2, 10, 1, matrix)

    # Extract the traceback object
    tb = result.traceback

    # Get aligned sequences
    # a1 = tb.query.replace("-", "<GAP>")
    # a2 = tb.ref.replace("-", "<GAP>")
    a1 = tb.query
    a2 = tb.ref

    return a1, a2

# Test sequences
seq1 = "MKTAYIAKQRQISFVKSHFSRQLEERLGLIE"
seq2 = "MKTVYIAKRRQISFLKSHFSRQLDDRLGLIE"

a1, a2 = align_pair(seq1, seq2)

print("Seq1 Aligned:", a1)
print("Seq2 Aligned:", a2)


Seq1 Aligned: MKTAYIAKQRQISFVKSHFSRQLEERLGLIE
Seq2 Aligned: MKTVYIAKRRQISFLKSHFSRQLDDRLGLIE


In [ ]:
from gLM.sequences import align_pair
seq1 = "MKTAYIAKQRQISFVKSHFS"
seq2 = "MKTA---QRQISFVKSHFSSS"

a1, a2 = align_pair(seq1, seq2)

print("Seq1 Aligned:", a1)
print("Seq2 Aligned:", a2)

print(f"Seq1 Aligned with [GAP]:", a1.replace("-", "[GAP]"))
print(f"Seq2 Aligned with [GAP]:", a2.replace("-", "[GAP]"))

Seq1 Aligned: MKTAYIAKQRQISFVKSHF--S
Seq2 Aligned: MKTA----QRQISFVKSHFSSS
Seq1 Aligned with [GAP]: MKTAYIAKQRQISFVKSHF[GAP][GAP]S
Seq2 Aligned with [GAP]: MKTA[GAP][GAP][GAP][GAP]QRQISFVKSHFSSS


In [ ]:
from transformers import (
    PreTrainedTokenizerFast,
)

class PhyloTokenizerLoader:
    def __init__(self, tokenizer_path):
        self.tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_path)
    
    def encode(self, aligned_seq1, aligned_seq2):
        inputs = self.tokenizer(
            aligned_seq2,
            add_special_tokens=False
        )
        labels = self.tokenizer(
            aligned_seq1,
            add_special_tokens=False
        )
        return{
            "input_ids": inputs["input_ids"],
            "labels": labels["input_ids"], 
            "attention_mask": inputs["attention_mask"]
        }
    
    def __call__(self, *args, **kwargs):
        return self.tokenizer(*args, **kwargs)

# Set breakpoint inside encode() method
# Then run this:
phylo_tokenizer = PhyloTokenizerLoader("phylo_char_tokenizer")

# When you call encode, debugger will pause at your breakpoint
result = phylo_tokenizer.encode("ATCG", "ATCG")

In [ ]:
result

{'input_ids': [6, 22, 7, 11],
 'labels': [6, 22, 7, 11],
 'attention_mask': [1, 1, 1, 1]}

In [ ]:
from gLM.dataset import UniRefClusterIterableDataset
from gLM.tokenizers import PhyloTokenizerLoader


In [ ]:
tokenizer = PhyloTokenizerLoader("./phylo_char_tokenizer").load()


In [ ]:
ds = UniRefClusterIterableDataset(
    parquet_path="/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet",
    fasta_path="/gpfs/data/brandeslab/Data/uniref/uniref100.fasta",
    tokenizer=tokenizer,
    max_seq_len=8192,
)



TypeError: UniRefClusterIterableDataset.__init__() missing 1 required positional argument: 'index_db_path'

In [ ]:
for i, x in enumerate(ds):
    print("Got item", i)
    if i == 2:
        break


In [ ]:

import time
from gLM.dataset import UniRefClusterIterableDataset
from gLM.tokenizers import PhyloTokenizerLoader

parquet = "/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet"
fasta   = "/gpfs/data/brandeslab/Data/uniref/uniref100.fasta"
index_db_path = "/gpfs/data/brandeslab/User/as12267/uniref100.idx"

print("Loading tokenizer...")
tok = PhyloTokenizerLoader("./phylo_char_tokenizer").load()

print("Instantiating dataset...")
start = time.time()
ds = UniRefClusterIterableDataset(
    parquet_path=parquet,
    index_db_path=index_db_path,
    fasta_path=fasta,
    tokenizer=tok,
    max_seq_len=8192,
    training_type="MLM"
)
print("Dataset constructed in", time.time() - start, "sec")

print("Fetching first item...")
start = time.time()
for i, item in enumerate(ds):
    print("Got first item!")
    break
print("First item fetched in", time.time() - start, "sec")



Loading tokenizer...
Instantiating dataset...
Dataset constructed in 0.04993486404418945 sec
Fetching first item...
In the Uniref Iterable - MLM branch
Got first item!
First item fetched in 0.4544064998626709 sec


In [ ]:
from gLM.dataset import UniRefClusterIterableDataset
from gLM.tokenizers import PhyloTokenizerLoader
import time

parquet = "/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet"
fasta   = "/gpfs/data/brandeslab/Data/uniref/uniref100.fasta"
index_db = "/gpfs/data/brandeslab/User/as12267/uniref100.idx"

print("Loading tokenizer...")
tok = PhyloTokenizerLoader("./phylo_char_tokenizer")

ds = UniRefClusterIterableDataset(
    parquet_path=parquet,
    index_db_path=index_db,
    fasta_path=fasta,
    tokenizer=tok,
    max_seq_len=8192,
)

print("Fetching first item...")
for i, x in enumerate(ds):
    print("Success!")
    print(x.keys())
    break



/gpfs/home/as12267/.conda/envs/huggingface_bert/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading tokenizer...
Fetching first item...
REP: A0ABS9P3Y9_9GAMM MEMBERS: [] <class 'list'>
REP: A0A835XQ69_9CHLO MEMBERS: ['A0A835XMW2_9CHLO'] <class 'list'>
PAIR: A0A835XQ69_9CHLO A0A835XMW2_9CHLO
REP: UPI000C86E4D3 MEMBERS: [] <class 'list'>
REP: UPI001260CD7F MEMBERS: [] <class 'list'>
REP: A0A2I1HB31_9GLOM MEMBERS: [] <class 'list'>
REP: A0AAF0EAU2_9BASI MEMBERS: [] <class 'list'>
REP: UPI002ECFBE8A MEMBERS: [] <class 'list'>
REP: UPI0002491D06 MEMBERS: [] <class 'list'>
REP: UPI002625D505 MEMBERS: [] <class 'list'>
REP: A0A917ZIC9_9GAMM MEMBERS: [] <class 'list'>
REP: UPI00188B8639 MEMBERS: [] <class 'list'>
REP: UPI003216E2CD MEMBERS: [] <class 'list'>
REP: A0A1R1K1Z6_ALCXX MEMBERS: ['UPI0029A3963B', 'UPI0005F94960'] <class 'list'>
PAIR: UPI0005F94960 A0A1R1K1Z6_ALCXX
REP: A0ABU7LKA1_9NOCA MEMBERS: ['UPI000B257CD9', 'UPI00364A542B'] <class 'list'>
PAIR: UPI000B257CD9 A0ABU7LKA1_9NOCA
REP: UPI00112C5A48 MEMBERS: [] <class 'list'>
REP: A0A1I2CAG8_9RHOB MEMBERS: [] <class 'list'>


In [ ]:
from gLM.tokenizers import PhyloTokenizerLoader

tok = PhyloTokenizerLoader("./phylo_char_tokenizer")

print(tok)                    # should show PreTrainedTokenizerFast
print(tok.mask_token)         # should NOT raise error
print(tok.pad_token_id)
print(tok.convert_tokens_to_ids("-"))
print(tok.get_vocab())
print(tok("ACD-EFG"))


/gpfs/home/as12267/.conda/envs/huggingface_bert/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[MASK]
0
5
{'[UNK]': 1, 'C': 7, '[MASK]': 4, 'T': 22, '[SEP]': 3, '[CLS]': 2, 'H': 12, 'E': 9, 'L': 15, 'F': 10, 'G': 11, 'Y': 25, '[PAD]': 0, 'Q': 19, 'K': 14, 'A': 6, '-': 5, 'P': 18, 'I': 13, 'W': 24, 'R': 20, 'D': 8, 'S': 21, 'M': 16, 'N': 17, 'V': 23}
{'input_ids': [6, 7, 8, 5, 9, 10, 11], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}


In [ ]:
tokenizer = PhyloTokenizerLoader("./phylo_char_tokenizer_updated")
pad_id = tokenizer.pad_token_id
mask_id = tokenizer.mask_token_id
non_gap_id = tokenizer.convert_tokens_to_ids("-") 
gap_id = tokenizer.convert_tokens_to_ids("[GAP]")
print("Phylo Tokenizer loaded")
print("Mask ID:", mask_id)
print("Non-GAP ID:", non_gap_id)
print("GAP ID:", gap_id)
print("Tokenizer vocab size:", tokenizer.vocab_size)

Phylo Tokenizer loaded
Mask ID: 4
Non-GAP ID: 6
GAP ID: 5
Tokenizer vocab size: 27


In [ ]:
from gLM.dataset import UniRefClusterIterableDataset
from gLM.tokenizers import PhyloTokenizerLoader
from gLM.collator import SequencePairCollator
import time
from torch.utils.data import DataLoader

parquet = "/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet"
fasta   = "/gpfs/data/brandeslab/Data/uniref/uniref100.fasta"
index_db = "/gpfs/data/brandeslab/User/as12267/uniref100.idx"

print("Loading tokenizer...")
# tok = PhyloTokenizerLoader("./phylo_char_tokenizer")
tok = PhyloTokenizerLoader("./phylo_char_tokenizer_updated")

ds = UniRefClusterIterableDataset(
    parquet_path=parquet,
    index_db_path=index_db,
    fasta_path=fasta,
    tokenizer=tok,
    max_seq_len=8192,
    training_type="phylo"
)


collator = SequencePairCollator(pad_id=tok.tokenizer.pad_token_id)

loader = DataLoader(
    ds, 
    batch_size=4, 
    collate_fn=collator,
    num_workers=0

)
batch = next(iter(loader))

print("input_ids shape:", batch["input_ids"].shape)
print("labels shape:", batch["labels"].shape)
print("attention_mask shape:", batch["attention_mask"].shape)
print("percent_identity:", batch["percent_identity"])

/gpfs/home/as12267/.conda/envs/huggingface_bert/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading tokenizer...
In the Uniref Iterable - Phylo branch
In the Uniref Iterable - Phylo branch
In the Uniref Iterable - Phylo branch
Untokenize seq A1 MDTEGFPRQFARTRRFSLGVPREFTVSPDGDRVLFLRSESGVVPRVHLWMYESGGERILTDPA[GAP][GAP][GAP][GAP][GAP][GAP][GAP][GAP][GAP][GAP][GAP][GAP][GAP][GAP][GAP][GAP][GAP][GAP][GAP][GAP][GAP][GAP][GAP]AGVGTYATDRHVRVVAYTVDGSLWTVRTDGGLPRRIQTVGPVRDPRPSPDGTLIAYVTGGALRVVGTDGAGDRPLAEPETAETTYGLADYSAVASIGRSRGYWWSPDSGALLVARVDTSVVERRYLSDPSDPGQSPRSVRYPAAGTANAITSLHLVTVAGGHTPVRLPRQAPAKDAPADSWGLAFEYVVGADWQSGGPVISLQTRDQRTMWVLRVDPVHGSVEQLSRQTDQGWVEFPPGAPLHTASGVLVLPQVRGDERTIRIGGVFAPAGLQVRAVLGSVGEKVWFAASEEPTEVHVWSYEAGHGFERLTQTPGVHTATAGGDTLVLDSRTLGGHAVTVLRDGKQVGCIAVLAEQPLVTPRPVHLTLGKRQLRSRLHLPSWYEPGMAKLPVLLSPYAGPGMQVVTKAHGWYTAVCQWYAEQGFAVLATDGRGTPGRGVRWQRAILGDRLTPVLDDQIDGLHAAARRCDALDLERVGIRGWSFSGYLAAGAVLHRPDVFHAAVAGATPTDRRLYDTYWEERFLGHPDLQPHNYERSSLLPLAEKLTRPLMLVHGLADDNVAPAHTLRLSAELLAAGRPHRVLLLPGVGHLVTGEGVADTLLQLELDFLKSSLGA, length A1: 792
Untokenize length A1: 792
Untokenize s

In [ ]:
import time
from torch.utils.data import DataLoader
from gLM.dataset import UniRefClusterIterableDataset
from gLM.tokenizers import PhyloTokenizerLoader
from gLM.collator import SequencePairCollator

# === File paths ===
parquet = "/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet"
fasta   = "/gpfs/data/brandeslab/Data/uniref/uniref100.fasta"
index_db = "/gpfs/data/brandeslab/User/as12267/uniref100.idx"

# === Load tokenizer ===
print("Loading tokenizer...")
tok = PhyloTokenizerLoader("./phylo_char_tokenizer_updated")

# === Dataset ===
ds = UniRefClusterIterableDataset(
    parquet_path=parquet,
    index_db_path=index_db,
    fasta_path=fasta,
    tokenizer=tok,
    max_seq_len=8192,
    training_type="phylo",  # or "MLM"
    batch_size=128  # this is internal batch collection, not DataLoader batch_size
)

# === Collator ===
collator = SequencePairCollator(tok)

# === Dataloader ===
loader = DataLoader(
    ds,
    batch_size=128,  # this is the final batch size the model sees
    collate_fn=collator,
    num_workers=16,  # Use >0 to test multiprocessing
    pin_memory=True
)

# === Test: Get one batch ===
start = time.time()
batch = next(iter(loader))
end = time.time()

# === Inspect batch ===
print("\n--- Batch Info ---")
print("input_ids shape:     ", batch["input_ids"].shape)
print("labels shape:        ", batch["labels"].shape)
print("attention_mask shape:", batch["attention_mask"].shape)
print("percent_identity:    ", batch["percent_identity"])
print(f"Loaded 1 batch in {end - start:.3f} seconds")


Loading tokenizer...



--- Batch Info ---
input_ids shape:      torch.Size([128, 2198])
labels shape:         torch.Size([128, 2198])
attention_mask shape: torch.Size([128, 2198])
percent_identity:     tensor([80.3828, 98.7730, 98.1013, 98.5714, 96.6057, 95.7672, 99.6068, 92.6471,
        99.9545, 99.0654, 96.6728, 98.1366, 99.7462, 98.2507, 92.7602, 95.2381,
        99.0826, 99.3644, 90.8257, 92.8158, 96.7914, 94.7917, 99.2924, 93.0000,
        98.7730, 75.1196, 93.1452, 96.5517, 90.3509, 99.9168, 99.4429, 96.6346,
        91.4286, 99.0937, 97.2414, 91.8841, 89.7674, 92.0489, 94.0887, 98.4375,
        91.1565, 99.3243, 88.2129, 99.0991, 92.6471, 97.5510, 97.5845, 99.5968,
        93.1559, 98.7730, 80.6452, 98.8166, 98.1183, 91.3979, 88.0769, 91.3043,
        97.8261, 99.4012, 74.9231, 99.4220, 93.5484, 99.6689, 99.3072, 90.2017,
        90.1408, 94.3114, 92.0038, 99.3088, 90.7990, 89.1374, 94.8718, 95.0000,
        99.7630, 98.9305, 97.0588, 95.5823, 99.6805, 96.9595, 91.1504, 90.5028,
        93.4138, 92.

In [ ]:
tokenizer = PhyloTokenizerLoader("./phylo_char_tokenizer").load()
pad_id = tokenizer.pad_token_id
gap_id = tokenizer.convert_tokens_to_ids("-")
print("Phylo Tokenizer loaded. GAP ID:", gap_id)
print(f"vocab size: {tokenizer.vocab_size}")

AttributeError: PreTrainedTokenizerFast has no attribute load

In [ ]:
from gLM.data_utils.uniref_cluster_sampler import RandomClusterSampler

sampler = RandomClusterSampler(parquet)
for _ in range(5):
    cluster = sampler.sample_clusters()
    print(cluster)


{'cluster_id': 'UniRef90_A0AAD2JM23', 'representative_id': 'A0AAD2JM23_9STRA', 'members': ['A0A9K3LU73_9STRA']}
{'cluster_id': 'UniRef90_A0A2T5GHF6', 'representative_id': 'A0A2T5GHF6_9SPHN', 'members': ['UPI0011B1CF18', 'UPI00177CA10F', 'UPI00335862B2', 'UPI001783B8B9', 'UPI00178503EB', 'UPI002A6B860E']}
{'cluster_id': 'UniRef90_A0ABD3RNB6', 'representative_id': 'A0ABD3RNB6_9LAMI', 'members': []}
{'cluster_id': 'UniRef90_A0ABN7WPN7', 'representative_id': 'A0ABN7WPN7_GIGMA', 'members': ['A0ABN7X348_GIGMA']}
{'cluster_id': 'UniRef90_UPI0031F96DD0', 'representative_id': 'UPI0031F96DD0', 'members': []}


In [ ]:
import parasail
print(parasail.__version__)


1.3.4


In [1]:
from gLM.tokenizers import PhyloTokenizerLoader

tokenizer = PhyloTokenizerLoader("./phylo_char_tokenizer_updated")
pad_id = tokenizer.pad_token_id
gap_id = tokenizer.convert_tokens_to_ids("-")
print("Phylo Tokenizer loaded. pad ID:", pad_id)
print("Phylo Tokenizer loaded. GAP ID:", gap_id)
print(f"vocab size: {tokenizer.vocab_size}")

/gpfs/home/as12267/.conda/envs/huggingface_bert/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Phylo Tokenizer loaded. pad ID: 0
Phylo Tokenizer loaded. GAP ID: 6
vocab size: 27


In [2]:
from gLM.dataset import SeqPairIterableDataset
from gLM.tokenizers import PhyloTokenizerLoader
from gLM.collator import PhyloCollator
from gLM.train_utils import PhyloTrainer

train_file = "/gpfs/data/brandeslab/Data/uniref/hf_pairs_uniref90_final/train.jsonl"
train_ds = SeqPairIterableDataset(dataset_path=train_file,
                                  tokenizer=tokenizer, 
                                  training_type="phylo_encoder_decoder", 
                                  shuffle_buffer=100000)
print(train_ds)

In [3]:
collator = PhyloCollator(tokenizer=tokenizer, 
                         training_type ="phylo_encoder_decoder", 
                         max_seq_len = 8192)



In [4]:
from transformers import TrainingArguments
from gLM.train_utils import PhyloTrainer
from gLM.models import ProteinT5Model


args = TrainingArguments(
    output_dir="./debug_phylo",
    per_device_train_batch_size=4,   
    gradient_accumulation_steps=1,
    max_steps=2,                     
    logging_steps=1,
    save_steps=0,
    report_to="none")                 

model = ProteinT5Model(
            vocab_size=tokenizer.vocab_size, 
            tokenizer=tokenizer
        ).build()

model.config.decoder_start_token_id = tokenizer.pad_token_id
print("decoder_start_token_id =", model.config.decoder_start_token_id)
model.gradient_checkpointing_enable()
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
print("Hidden size  used:", model.config.d_model)

trainer = PhyloTrainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    data_collator=collator,
    tokenizer=tokenizer,
)


/tmp/ipykernel_2484957/1944515737.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `PhyloTrainer.__init__`. Use `processing_class` instead.
  trainer = PhyloTrainer(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


decoder_start_token_id = 0
Model parameters: 44,070,912
Hidden size  used: 512


In [5]:
import torch
dl = trainer.get_train_dataloader()
batch = next(iter(dl))

for k, v in batch.items():
    if torch.is_tensor(v):
        print(f"{k}: shape={v.shape}, dtype={v.dtype}")
        print("-100 counts:", (v == -100).sum().item())
    else:
        print(f"{k}: {v}")


input_ids shape: torch.Size([4, 2500])
attention_mask shape: torch.Size([4, 2500])
attention_mask sum: tensor([ 768, 1501, 2500,  722])
seq lengths in labels tensor([ 768, 1350, 2609,  722])
labels shape: torch.Size([4, 2609])
input_ids: shape=torch.Size([4, 2500]), dtype=torch.int64
-100 counts: 0
attention_mask: shape=torch.Size([4, 2500]), dtype=torch.int64
-100 counts: 0
labels: shape=torch.Size([4, 2609]), dtype=torch.int64
-100 counts: 4987


In [4]:
import torch
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_ds,
    batch_size=4,
    collate_fn=collator,
    pin_memory=True
)

for batch in train_loader:
    # print("input_ids:", batch["input_ids"].shape)
    # print("attention_mask:", batch["attention_mask"].shape)
    # print("labels:", batch["labels"].shape)
    break

[0] Target raw: 'MTEVFLLGRRRPEENIAGANTMGQTANEDTFDWDKYLKETGSVSAPSEYFRQSKTPPTNEFQIGMKLEARDPRNIDSMCVASVIGITGARLRLRLDGSDNKNDFWRLVDSSDIQPVGTCEQLGDLLQPPLGYRMNASSWPMFLLRILTGSELAPAVFFKEEPPRPPQNNFIVGMKIEAVDRKNPFMICPATIGAVSGDQIHITFDGWSGAFDYWCDYDSRDIFPVGWCRLTGDVLQPPGKVVEKRPRAKAVTRIWRLRNMFLENTEAAPEAAEKPGCSSFTTTSGDEDRTLEDYQAAPAGKKPRGRGFTKPLEDEVRPGRDVQVAPVEEKTKGKRVTKPWKDQATPFEDEAAAPAEKKRKSKTVTTPWTDQPRLFADQGTAPAEKKRKGKTVTTPWSDQPQLFADQGAAPAEKKHKGKTFATPWTDQPRLFADQGAAPAEKKRKGKTVTTPWTDQPRLFADQGAAPAEKKRKGKTVTTPWIDQPRLFADQGAAPAEKKRKGKTVTTSWTDQPRLFADQGAAPGEKKRKGKTVTTPWSDQPRLFADQGAAPGEKKRKGKTVTTPWSDQPRLFADQGAAPAEKKHKGKTVTTSWTDQPRLFADQGAAPAEKKRKGKTFATPWTDQPRLFADQGAAPAEKKRKGKTVTTPWIDQPRLFAVQGTAPAEKKHKGKTVTTPWSDQPRLFADQGVAPAEKKPKGKRVTKPWKDQSIPFYFGVASPAEKKPKGKRVTKRQKDQAQFLADEEAMPALFSALSVSSTEKTPPSSSEQSKSSPSGKTSSTAKGAQSSRKSPRKTSVVQKVPKTSKKAGKTKSTKDTSSTKKGITIKIVLPKKRGGKSGKKEKSIPVVSSTSSASLSSLMKSSSKTPEEPSKIVMSTVCVYINKHGDCGPYLDPQKVQQLPSHFGPGPVNVILRQIVQACVDCAIVPKDVFLLLKPDNRGGEMITAFFDGKVNTVQLPPVNSASFALRFLENFCHSLNCDKFLSS

In [5]:
batch

{'input_ids': tensor([[17, 23, 10,  ...,  0,  0,  0],
         [17, 10, 14,  ...,  0,  0,  0],
         [17, 23,  7,  ..., 12, 25, 10],
         [17, 23, 12,  ...,  0,  0,  0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': tensor([[  17,   23,   10,  ..., -100, -100, -100],
         [  17,   10,   14,  ..., -100, -100, -100],
         [  17,   23,    7,  ...,   12,   25,   20],
         [  17,    7,   14,  ..., -100, -100, -100]])}

In [12]:
input_ids = batch["input_ids"]
attention_mask = batch["attention_mask"]
labels = batch["labels"]

print("input_ids shape:", input_ids.shape)
print("attention_mask shape:", attention_mask.shape)
print("labels shape:", labels.shape)

pad_id = tokenizer.pad_token_id
gap_id = tokenizer.convert_tokens_to_ids("[GAP]")
print("Pad ID:", pad_id)
print("GAP ID:", gap_id)
print("PAD count in input_ids", (input_ids == pad_id).sum().item()) # comes from seq 1
print("GAP count in input_ids", (input_ids == gap_id).sum().item()) # comes from seq 1
print("PAD count in labels", (labels == pad_id).sum().item()) # should be 0 
print("GAP count in labels", (labels == gap_id).sum().item()) # comes from seq 2
print("-100 count in labels", (labels == -100).sum().item()) # comes from seq 2

input_ids shape: torch.Size([4, 2500])
attention_mask shape: torch.Size([4, 2500])
labels shape: torch.Size([4, 2609])
Pad ID: 0
GAP ID: 5
PAD count in input_ids 4509
GAP count in input_ids 0
PAD count in labels 0
GAP count in labels 0
-100 count in labels 4987


In [23]:
print(len(batch["input_ids"][1]))
print(batch["input_ids"][1])

1437
tensor([17,  7, 14,  ...,  0,  0,  0])
